# Generate Toronto taxi and rideshare trips
## Overview
In this pipeline, we generate synthetic trips across the Greater Toronto Area (GTA) based on TTS 2016 trip counts between different wards at various times throughout a single day. The pipeline goes **Generate trip locations &rarr; Generate trip start times &rarr; Add person attributes**.

## Inputs
All data from 2016 TTS survey, where the 44-ward model was still in use. The core principle is this: use the distribution of trip counts, split across different attributes (location, time, income, age), to create a smaller, synthetic dataset with sampled attributes.

### TTS
Query Criteria:
- Tabulation type: Cross Tabulation
- Data type: Trip
- Survey Year: 2016
- Expansion Factor: ON

More query details are below in the notebook as we go.

### Utilities
* 44-ward boundaries shapefile: for sampling coordinates after we've sampled the origin and destination wards. Source: https://open.toronto.ca/dataset/city-wards/

## Outputs
A CSV file of trip information, income range, and age range.

In [1]:
import attribute_dist

import csv
from collections import namedtuple
from datetime import date, datetime, timedelta
from math import radians, cos, sin, asin, sqrt
import random

import geopandas as gpd
import numpy as np
import pandas as pd
import seaborn as sns
from shapely.geometry import Polygon, Point

In [2]:
np.random.seed(16)
random.seed(16)

# Part 1: Sample trip origin/destination

Filter variables:
- Row Attribute: Ward number of origin
- Column Attribute: Ward number of destination

Filter selection:
- Primary travel mode of trip `IN` T (Taxi passenger), U (Paid rideshare)

In [3]:
# Sample population size
N = 10000

# Bounding box for the scenario
Bbox = namedtuple("Bbox", ['x1', 'y1', 'x2', 'y2'])
bbox = Bbox(43.581006,-79.639268,43.855468,-79.115246)

# Restrict wards
wards = np.array(range(1, 45))

# Read O-D matrix
od_file = 'tts/od/taxi_rideshare.csv'
od_all = pd.read_csv(od_file, index_col=0).reindex()
od_all = od_all.loc[0:44, list(map(str, range(1, 45)))]
print("Total number of taxi/rideshare trips: %d" % od_all.sum().sum())

Total number of taxi/rideshare trips: 60221


## Generate origin ward
Take the sum along the columns of the table, so that we have the total number of trips that started in each ward. Normalize so we have a valid probability distribution and sample.

In [4]:
od_start = od_all.apply(np.sum, axis=1)
od_start = od_start / od_start.sum()
start_wards = np.array([np.random.choice(wards, p=od_start.values) for _ in range(N)])
print("Sampled %d origin wards" % N)

Sampled 10000 origin wards


## Generate destination ward
Similarly, get the frequency distribution for each origin ward and sample the destination ward for each origin ward chosen in the previous step.

In [5]:
od_end = od_all.divide(od_all.apply(np.sum, axis=1).values, axis=0)
end_wards = np.array([np.random.choice(wards, p=od_end.iloc[s-1].values) for s in start_wards])
print("Sampled %d destination wards" % N)
od_pairs = [*zip(start_wards, end_wards)]
od_pairs[0:10]

Sampled 10000 destination wards


[(19, 14),
 (25, 27),
 (26, 40),
 (6, 14),
 (20, 28),
 (19, 27),
 (27, 32),
 (16, 25),
 (8, 8),
 (37, 37)]

## Sampling coordinates inside a ward
Given the ward, fetch the ward's polygon bounds from the .shp file, then sample a lat-long coordinate within this polygon.

In [6]:
ward_locations = gpd.read_file('wards/icitw_wgs84.shp')
ward_locations

,GEO_ID,CREATE_ID,NAME,SCODE_NAME,LCODE_NAME,TYPE_DESC,TYPE_CODE,OBJECTID,SHAPE_AREA,SHAPE_LEN,geometry
0,14630026,63519,Scarborough-Rouge River (41),41,EA41,Ward,CITW,1,0.0,0.0,"POLYGON ((-79.26486 43.77956, -79.26495 43.779..."
1,14630028,63519,Scarborough East (44),44,EA44,Ward,CITW,2,0.0,0.0,"POLYGON ((-79.17077 43.75564, -79.17080 43.755..."
2,14630024,63519,Scarborough-Rouge River (42),42,EA42,Ward,CITW,3,0.0,0.0,"POLYGON ((-79.22568 43.78940, -79.22579 43.789..."
3,14630027,63519,Scarborough-Agincourt (39),39,EA39,Ward,CITW,4,0.0,0.0,"POLYGON ((-79.33142 43.79312, -79.33183 43.794..."
4,14630035,63519,Willowdale (24),24,NO24,Ward,CITW,5,0.0,0.0,"POLYGON ((-79.38720 43.76348, -79.38726 43.763..."
5,14630029,63519,Scarborough-Agincourt (40),40,EA40,Ward,CITW,6,0.0,0.0,"POLYGON ((-79.31525 43.75788, -79.31563 43.758..."
6,14630036,63519,Don Valley East (33),33,NO33,Ward,CITW,7,0.0,0.0,"POLYGON ((-79.31989 43.76857, -79.32035 43.768..."
7,14630037,63519,Willowdale (23),23,NO23,Ward,CITW,8,0.0,0.0,"POLYGON ((-79.40844 43.75382, -79.41166 43.751..."
8,14630039,63519,York West (8),08,NO08,Ward,CITW,9,0.0,0.0,"POLYGON ((-79.51513 43.74711, -79.51569 43.749..."
9,14630031,63519,Scarborough Centre (38),38,EA38,Ward,CITW,10,0.0,0.0,"POLYGON ((-79.24809 43.73671, -79.24825 43.736..."


In [7]:
ward_polygons = ward_locations.loc[:, ['SCODE_NAME', 'geometry']].set_index('SCODE_NAME')
ward_polygons.index = ward_polygons.index.astype(int)
ward_polygons = ward_polygons.to_dict()['geometry']

# Print the maximum bounds
print(ward_polygons[42].bounds) # maxy
print(ward_polygons[44].bounds) # maxx
print(ward_polygons[1].bounds)  # minx
print(ward_polygons[6].bounds)  # miny

(-79.25884175715663, 43.781453980370074, -79.15208584581502, 43.85546811779826)
(-79.19735903656243, 43.75563765091391, -79.11524635059097, 43.8146256926031)
(-79.63926825157114, 43.72252741697897, -79.54610481696473, 43.76302899644202)
(-79.55978538683242, 43.581006413660994, -79.46913008060552, 43.633249552476926)


In [8]:
def sample_point(polygon, ward=None):
    # Get ward bounds
    minx, miny, maxx, maxy = polygon.bounds
    
    # Special check for the waterfront ward so we don't end up in the lake
    if ward:
        if ward == 28:
            minx, miny = -79.405111, 43.639481
        elif ward == 30:
            minx, miny = -79.354584, 43.647710
            
    x, y = random.uniform(minx, maxx), random.uniform(miny, maxy)
    while not polygon.contains(Point(x, y)):
        x, y = random.uniform(minx, maxx), random.uniform(miny, maxy)

    # Return as lat, long
    return y, x

In [9]:
od_point_pairs = [sample_point(ward_polygons[o], ward=o) +\
                  sample_point(ward_polygons[d], ward=d) for o, d in od_pairs]
od_data = [a + b for a, b in zip(od_pairs, od_point_pairs)]
trips = pd.DataFrame(od_data,
                     columns=['origin_ward', 'dest_ward', 'origin_lat', 'origin_long', 'dest_lat', 'dest_long'])
trips

,origin_ward,dest_ward,origin_lat,origin_long,dest_lat,dest_long
0,19,14,43.648450,-79.420951,43.646540,-79.446202
1,25,27,43.747045,-79.380494,43.678952,-79.388040
2,26,40,43.703825,-79.334429,43.794631,-79.317442
3,6,14,43.615874,-79.486569,43.640889,-79.436559
4,20,28,43.668033,-79.410445,43.670878,-79.367937
...,...,...,...,...,...,...
9995,20,27,43.656032,-79.391506,43.655210,-79.381469
9996,19,18,43.632739,-79.416174,43.655609,-79.435230
9997,20,19,43.638571,-79.390953,43.629108,-79.421827
9998,40,40,43.797194,-79.309887,43.771408,-79.312521


In [10]:
trips_filtered = trips.loc[lambda df: (df['origin_lat'] > bbox.x1) & (df['dest_lat'] > bbox.x1) \
       & (df['origin_lat'] < bbox.x2) & (df['dest_lat'] < bbox.x2) \
       & (df['origin_long'] > bbox.y1) & (df['dest_long'] > bbox.y1) \
       & (df['origin_long'] < bbox.y2) & (df['dest_long'] < bbox.y2)]

print("Trips after filtering bounding box: %d" % len(trips_filtered))

Trips after filtering bounding box: 10000


# Part 2: Sample trip start timestamps

For each origin ward $w_o$, I download a count table of destination ward and start time of trip. Normalizing the counts to get a probability distribution, we randomly sample a start time for a given $(w_o, w_d)$ pair.

Filter variables:
- Row Attribute: Start time of trip
- Column Attribute: Ward number of destination

Filter selection:
- Primary travel mode of trip `IN` T, U
- Ward number of origin `IN` $w_o$

In [11]:
def sample_start_timeslot(o, d):
    o, d = int(o), int(d)
    fname = "tts/starttime/starttime_oward_{}_taxi_rideshare.csv".format(o)
    
    # Get the corresponding frequency distribution of start times for this o-d pair
    distribution = pd.read_csv(fname, index_col=0)[str(d)]
    
    # Convert to probability distribution
    probabilities = distribution.values / distribution.values.sum()
    
    # Sample a timeslot
    timeslot = np.random.choice(distribution.index, p=probabilities)
    return timeslot
    
def sample_start_time(timeslot, datestr):
    # Sample a random time inside that 5-minute timeslot
    time = random.randint(timeslot, timeslot + 5)
    
    # Format datetime string
    hours = int(time / 100)
    minutes = int(time) % 100
    seconds = int((time - int(time)) * 60)
    result = datetime.fromisoformat(datestr) + timedelta(hours=hours, minutes=minutes, seconds=seconds)
    
    return result.isoformat(sep='T', timespec='milliseconds')

In [12]:
DATE = "2020-09-20"
trips_filtered['trip_start_timeslot'] = trips_filtered.apply(
    lambda x: sample_start_timeslot(x['origin_ward'], x['dest_ward']),
    axis=1)
trips_filtered['trip_start_timestamp'] = trips_filtered.apply(
    lambda x: sample_start_time(x['trip_start_timeslot'], DATE),
    axis=1)

# Index in the order of trip_start_timestamp
trips_filtered.sort_values('trip_start_timestamp', inplace=True)
trips_filtered.reset_index(drop=True, inplace=True)

print("Sampled trip start times")
trips_filtered.head()

Sampled trip start times


,origin_ward,dest_ward,origin_lat,origin_long,dest_lat,dest_long,trip_start_timeslot,trip_start_timestamp
0,20,27,43.643036,-79.393776,43.692108,-79.382295,400,2020-09-20T04:01:00.000
1,19,11,43.645547,-79.409359,43.706870,-79.525427,400,2020-09-20T04:02:00.000
2,20,27,43.668058,-79.413465,43.662591,-79.375573,400,2020-09-20T04:02:00.000
3,19,11,43.634028,-79.412486,43.671505,-79.511795,400,2020-09-20T04:02:00.000
4,20,20,43.641935,-79.398009,43.660127,-79.393329,400,2020-09-20T04:05:00.000


# Part 3: Calculate trip distance, duration, and end timestamp

Trip distance and duration are not actually that useful for the simulation, but they provide a sanity check for the trips. During scenario creation we'll filter out trips that are too long or short. Haversine distance is used for the trip distance, and the duration is calculated by assuming a constant speed of 60 km/h.

In [13]:
def haversine_distance(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    
    # Mean radius of earth in km
    r = 6371
    return c * r

def simple_duration(dist, kph):
    return (dist / kph) * 60

def trip_endtime(start_time, duration):
    minutes, seconds = duration, (duration % 1) * 60
    end_time = datetime.fromisoformat(start_time) + timedelta(minutes=minutes, seconds=seconds)
    return end_time.isoformat(sep='T', timespec='milliseconds')

In [14]:
default_speed = 50

trips_filtered['distance'] = trips_filtered.apply(
    lambda x: haversine_distance(x['origin_long'], x['origin_lat'],
                                 x['dest_long'], x['dest_lat']),
    axis=1)
trips_filtered['duration'] = trips_filtered.apply(
    lambda x: simple_duration(x['distance'], 50),
    axis=1)
trips_filtered['trip_end_timestamp'] = trips_filtered.apply(
    lambda x: trip_endtime(x['trip_start_timestamp'], x['duration']),
    axis=1)
trips_filtered['trip_id'] = pd.Series(range(len(trips_filtered)))

# Part 4: Sample income and age attributes

There are 6 income groups: 0-15K, 15-40K, 40-60K, 60-100K, 100-125K, 125K+, and UNK (unknown), and I divide age into 7 ranges: 11-20, 21-30, 31-40, 41-50, 51-60, 61-70, 71+, and UNK (unknown). The attribute sampling process uses common filter variables for both age and income.

Filter variables:
- Row Attribute: Start time of trip
- Column Attribute: Ward number of destination
- Additional Attribute: Ward number of origin

Filter selection:
- Primary travel mode of trip `IN` T, U
- Ward number of origin `IN` 1-44
- Ward number of destination `IN` 1-44
- (AGE) Age of person `IN` <Age range\>
- (INCOME) Income range of household `IN` <Income group\>

Preprocessing of the query results to obtain the count tables is done in the `attribute_dist.py` file.

In [15]:
def sample_group(dist, groups, o, d, timeslot):    
    # Get trip counts
    counts = dist[o].at[timeslot, d]
    
    # Convert to probabilities
    prob = np.array(counts) / sum(counts)
    
    # Sample a group
    return np.random.choice(groups, p=prob)

dists = attribute_dist.AttributeDistributions()

## Age attributes

In [16]:
age_dist = dists.get_distributions('tts/age', range(1, 9))
age_groups = ['11-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71+', 'UNK']
trips_filtered['age_range'] = trips_filtered.apply(
    lambda x: sample_group(age_dist, age_groups,
                           x['origin_ward'], x['dest_ward'], x['trip_start_timeslot']),
    axis=1)

## Income attributes

In [17]:
income_dist = dists.get_distributions('tts/income', range(1, 8))
income_groups = ['0-15K', '15-40K', '40-60K', '60-100K', '100-125K', '125K+', 'UNK']
trips_filtered['income_range'] = trips_filtered.apply(
    lambda x: sample_group(income_dist, income_groups,
                           x['origin_ward'], x['dest_ward'], x['trip_start_timeslot']),
    axis=1)

In [18]:
trips_final = trips_filtered.loc[:,[
    'trip_id',
    'trip_start_timeslot',
    'trip_start_timestamp',
    'trip_end_timestamp',
    'duration',
    'distance',
    'origin_ward',
    'dest_ward',
    'origin_lat',
    'origin_long',
    'dest_lat',
    'dest_long',
    'age_range',
    'income_range']]

trips_final.head()

,trip_id,trip_start_timeslot,trip_start_timestamp,trip_end_timestamp,duration,distance,origin_ward,dest_ward,origin_lat,origin_long,dest_lat,dest_long,age_range,income_range
0,0,400,2020-09-20T04:01:00.000,2020-09-20T04:08:16.918,6.640986,5.534155,20,27,43.643036,-79.393776,43.692108,-79.382295,21-30,100-125K
1,1,400,2020-09-20T04:02:00.000,2020-09-20T04:16:44.607,13.871726,11.559772,19,11,43.645547,-79.409359,43.706870,-79.525427,21-30,40-60K
2,2,400,2020-09-20T04:02:00.000,2020-09-20T04:06:27.549,3.729575,3.107979,20,27,43.668058,-79.413465,43.662591,-79.375573,21-30,100-125K
3,3,400,2020-09-20T04:02:00.000,2020-09-20T04:13:37.626,10.813555,9.011296,19,11,43.634028,-79.412486,43.671505,-79.511795,21-30,40-60K
4,4,400,2020-09-20T04:05:00.000,2020-09-20T04:07:56.299,2.469165,2.057638,20,20,43.641935,-79.398009,43.660127,-79.393329,31-40,UNK


In [19]:
trips_final.to_csv('trips_final.csv', index=False)